# Quora Question pairs


Today it's a preparation for delivery 1.


The objective of today is to build a solution for:

https://www.kaggle.com/c/quora-question-pairs/overview

You might even make a late submission to kaggle to know how well you would have done in the competition.



In [1]:
import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np
import os

In [48]:
# use this to train and VALIDATE your solution
train_df = pd.read_csv("./quora_train_data.csv")

# use this to provide the expected generalization results
test_df = pd.read_csv("./quora_test_data.csv")

train_df

,id,qid1,qid2,question1,question2,is_duplicate
0,346692,38482,10706,Why do I get easily bored with everything?,Why do I get bored with things so quickly and ...,1
1,327668,454117,345117,How do I study for Honeywell company recruitment?,How do I study for Honeywell company recruitme...,1
2,272993,391373,391374,Which search engine algorithm is Quora using?,Why is Quora not using reliable search engine?,0
3,54070,82673,95496,How can I smartly cut myself?,Can someone who thinks about suicide for 7 yea...,0
4,46450,38384,72436,How do I see who is viewing my Instagram videos?,Can one tell who viewed my Instagram videos?,1
...,...,...,...,...,...,...
323427,192476,292119,292120,Is it okay to use a laptop while it is chargin...,Is it OK to use your phone while charging?,0
323428,17730,33641,33642,How can dogs understand human language?,Can dogs understand the human language?,0
323429,28030,52012,52013,What's your favourite lotion?,What's your favourite skin lotion?,1
323430,277869,397054,120852,How does one become a hedge fund manager?,What should I do to become a hedge fund manager?,1


In [6]:
A_df, te_df = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)

tr_df, va_df = sklearn.model_selection.train_test_split(A_df, test_size=0.05,random_state=123)

tr_df.shape= (291897, 6)
va_df.shape= (15363, 6)
te_df.shape= (16172, 6)


In [54]:
train_df.iloc[6106]

id                                        96725
qid1                                     161071
qid2                                     161072
question1                                     ?
question2       Who are Moses, Noah and Exodus?
is_duplicate                                  0
Name: 6106, dtype: object

# Improving feature vectors:

Build a Similarity metric between documents using the tf-idf vectors of the documents.

Build a class from scratch that can learn a tf-idf feature representation.

In [41]:
from utils import TfIdfCustomVectorizer, TfIdfEmbeddingVectorizer, preprocess
from gensim.models import Word2Vec

First we are going to train a Word2Vec model, so that it better fits our problem.

In [42]:
all_questions = list(train_df["question1"]) + list(train_df["question2"])

In [43]:
w2v_model = Word2Vec(preprocess(all_questions), vector_size=256, window=5, min_count=1, workers=4)
w2v_model.save('word2vec_quora.model')

In [44]:
from utils import TfIdfEmbeddingVectorizer

vectorizer = TfIdfEmbeddingVectorizer().fit(all_questions)

X = np.c_[
    vectorizer.transform(train_df['question1']),
    vectorizer.transform(train_df['question2'])
]
y = train_df['is_duplicate']


clf = ensemble.RandomForestClassifier()
clf.fit(X, y)

RandomForestClassifier()

In [45]:
y_hat = clf.predict(np.c_[vectorizer.transform(test_df['question1']), vectorizer.transform(test_df['question2'])])

In [46]:
print(metrics.classification_report(test_df['is_duplicate'],y_hat))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86     50868
           1       0.84      0.57      0.68     29990

    accuracy                           0.80     80858
   macro avg       0.82      0.76      0.77     80858
weighted avg       0.81      0.80      0.79     80858

